In [ ]:
! pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
qa_dataset = load_dataset("mpingale/mental-health-chat-dataset")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/840 [00:00<?, ?B/s]

data/train-00000-of-00001-991edb316b3098(…):   0%|          | 0.00/3.64M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

In [ ]:
# save_path = "/content/drive/MyDrive/loaded_dataset/dude-sample-qas"
# qa_dataset.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
# from datasets import load_from_disk
# qa_dataset = load_from_disk("/content/drive/MyDrive/loaded_dataset/dude-sample-qas")

In [ ]:
train_qas = qa_dataset["train"]

print(train_qas[0])

{'questionID': 0, 'questionTitle': 'Do I have too many issues for counseling?', 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling', 'topic': 'depression', 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor', 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari', 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be h

In [ ]:
qa_dataset["train"][0]["questionText"]

'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?'

In [ ]:
from collections import defaultdict
import json

In [ ]:
def prepare_dataset(dataset):
    # لكل سؤال هنجمع: العنوان، النص، وكل الإجابات
    merged = defaultdict(lambda: {
        "questionTitle": None,
        "questionText": None,
        "all_answers": []
    })

    for item in dataset:
        qid = item.get("questionID")
        answer = item.get("answerText")

        # تخطي أي عنصر ناقص QID أو Answer
        if qid is None or not answer:
            continue

        # حفظ العنوان والنص لأول مرة فقط
        if merged[qid]["questionTitle"] is None:
            merged[qid]["questionTitle"] = item.get("questionTitle")
            merged[qid]["questionText"] = item.get("questionText")

        # تنظيف الإجابة
        clean_answer = answer.replace("\xa0", " ").strip()

        # إضافة الإجابة لو مش مكررة
        if clean_answer not in merged[qid]["all_answers"]:
            merged[qid]["all_answers"].append(clean_answer)

    # تحويل الدكشنري لقائمة نهائية
    prepared_dataset = []
    for qid, v in merged.items():
        combined_answer = "\n\n---\n\n".join(v["all_answers"])  # فاصل بين الإجابات
        prepared_dataset.append({
            "questionID": qid,
            "questionTitle": v["questionTitle"],
            "questionText": v["questionText"],
            "combined_answer": combined_answer
        })

    return prepared_dataset


In [ ]:
# تشغيل الدالة
prepared_dataset = prepare_dataset(train_qas)

In [ ]:
# طباعة أول 5 عناصر
for item in prepared_dataset[:5]:
    print(json.dumps(item, indent=2, ensure_ascii=False))
    print("=" * 120)

{
  "questionID": 0,
  "questionTitle": "Do I have too many issues for counseling?",
  "questionText": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "combined_answer": "It is very common for people to have multiple issues that they want to (and need to) address in counseling.  I have had clients ask that same question and through more exploration, there is often an underlying fear that they  \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,  people have numerous significant stressors in their lives.  L

In [ ]:
final_dataset = []

for item in prepared_dataset:
    question = item["questionText"] or item["questionTitle"]
    answer = item["combined_answer"]

    final_dataset.append({
        "instruction": question.strip(),
        "response": answer.strip()
    })

# طباعة أول 5 عينات
import json
for sample in final_dataset[:5]:
    print(json.dumps(sample, indent=2, ensure_ascii=False))
    print("=" * 80)

{
  "instruction": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "response": "It is very common for people to have multiple issues that they want to (and need to) address in counseling.  I have had clients ask that same question and through more exploration, there is often an underlying fear that they  \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not,  people have numerous significant stressors in their lives.  Let's face it, life can be complicated! Therapists are completely ready and equipped to hand

In [ ]:
len(final_dataset)

937

# cleaning the dataset

In [ ]:
import re
import html

In [ ]:
def clean_text(text):
    if not text:
        return ""

    text = html.unescape(text)

    # إزالة الرموز والإيموجي
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)

    # إزالة HTML أو markdown links
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)

    # إزالة \xa0
    text = text.replace("\xa0", " ")

    # إزالة فواصل أو فواصل طويلة مثل --- أو ===
    text = re.sub(r'-{3,}|={3,}', ' ', text)

    # إضافة مسافة بعد النقطة لو ناقصة
    text = re.sub(r'\.(\w)', r'. \1', text)

    # إزالة المسافات الزائدة قبل علامات الترقيم
    text = re.sub(r'\s+([,.:;?!])', r'\1', text)

    # توحيد المسافات
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
cleaned_dataset = []

for item in final_dataset:
    q = item["instruction"]
    a = item["response"]

    cleaned_dataset.append({
        "instruction": clean_text(q),
        "response": clean_text(a)
    })


In [ ]:
import json

for sample in cleaned_dataset[:5]:
    print(json.dumps(sample, indent=2, ensure_ascii=False))
    print("=" * 80)


{
  "instruction": "I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac. I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years. I’ve never had counseling about any of this. Do I have too many issues to address in counseling?",
  "response": "It is very common for people to have multiple issues that they want to (and need to) address in counseling. I have had clients ask that same question and through more exploration, there is often an underlying fear that they \"can't be helped\" or that they will \"be too much for their therapist.\" I don't know if any of this rings true for you. But, most people have more than one problem in their lives and more often than not, people have numerous significant stressors in their lives. Let's face it, life can be complicated! Therapists are completely ready and equipped to handle all of t

In [ ]:
max_words = 0
max_item = None

for item in cleaned_dataset:
    # نعد الكلمات في الـ response
    word_count = len(item["response"].split())

    if word_count > max_words:
        max_words = word_count
        max_item = item

print("أكبر عدد كلمات:", max_words)
print("="*80)
print("العنصر صاحب أكبر إجابة:")
import json
print(json.dumps(max_item, indent=2, ensure_ascii=False))


أكبر عدد كلمات: 15371
العنصر صاحب أكبر إجابة:
{
  "instruction": "How do you know you have the right therapist for you? How would I know how to \"train\" my therapist to be able to give me what I need from treatment?",
  "response": "Finding the right therapist for you is very important and can sometimes be tricky. It can sometimes take a number of sessions to get a good sense of whether you and your therapist are the \"right fit.\" The first couple of sessions are generally spent on gathering information, formulating a plan of treatment, and building the client/therapist relationship. The client/therapist relationship will be very different from other relationships you have experienced. You will know you have found the right therapist when you notice there is a good rapport between the two of you. You will get a sense that the therapist \"gets you\" and understands the issues being presented. If you feel that you can trust your therapist and feel comfortable opening up and providing f

In [ ]:
# حساب طول كل إجابة
lengths = [(len(item["response"].split()), item) for item in cleaned_dataset]

# ترتيبهم تنازلي
lengths_sorted = sorted(lengths, key=lambda x: x[0], reverse=True)

# عرض أول 5
for count, item in lengths_sorted[:5]:
    print(f"عدد كلمات: {count}")
    print(json.dumps(item, indent=2, ensure_ascii=False))
    print("="*80)


عدد كلمات: 15371
{
  "instruction": "How do you know you have the right therapist for you? How would I know how to \"train\" my therapist to be able to give me what I need from treatment?",
  "response": "Finding the right therapist for you is very important and can sometimes be tricky. It can sometimes take a number of sessions to get a good sense of whether you and your therapist are the \"right fit.\" The first couple of sessions are generally spent on gathering information, formulating a plan of treatment, and building the client/therapist relationship. The client/therapist relationship will be very different from other relationships you have experienced. You will know you have found the right therapist when you notice there is a good rapport between the two of you. You will get a sense that the therapist \"gets you\" and understands the issues being presented. If you feel that you can trust your therapist and feel comfortable opening up and providing feedback during your sessions 

In [ ]:
import json

with open("cleaned_dataset.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_dataset, f, ensure_ascii=False, indent=2)

# Load The Model

In [ ]:
!pip install -q unsloth
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 22.7 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# اختار موديل من موديلات Unsloth على HF
MODEL_ID = "unsloth/Llama-3.2-3B-Instruct"  # غيّر للي تحبه من القائمة

max_seq_length = 2048  # كفاية للأسئلة والإجابات
dtype = None           # يختار الأنسب (fp16/bf16) حسب الGPU
load_in_4bit = True    # علشان يوفر VRAM في Colab

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_ID,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)  # مهم علشان inference سريع

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0): LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNo

In [ ]:
import torch

# تحديث دالة التوليد: نضمن إرسال attention_mask ونقصّ النتيجة بحيث نرجع بس الجزء المولَّد (مش prompt)
def generate_answer(question: str, max_new_tokens: int = 256, temperature: float = 0.2):
    prompt = (
        "You are a helpful, empathetic mental health assistant. "
        "You give supportive, safe, non-judgmental answers and encourage seeking professional help when needed.\n\n"
        f"User: {question}\nAssistant:"
    )

    # tokenization مع attention_mask
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # توليد مع تمرير attention_mask صراحة
    with torch.no_grad():
        out = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode: نص كامل = prompt + generated؛ نريد الجزء الجديد فقط
    generated_text = tokenizer.decode(out[0], skip_special_tokens=True)

    # لو عايز الجزء اللي بعد الـ prompt بس:
    # نحسب طول الـ prompt tokens
    prompt_len = input_ids.shape[1]
    generated_tokens = out[0][prompt_len:]
    generated_only = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

    return generated_only if generated_only else generated_text

In [ ]:
import json, time, traceback, os

sample_n = 5
results = []
path = "sample_comparison.json"

for idx, item in enumerate(cleaned_dataset[:sample_n]):
    qid = item.get("questionID") if item.get("questionID") is not None else f"idx_{idx}"
    question = item.get("instruction") or item.get("questionTitle") or ""
    gt = item.get("response")

    try:
        print(f"\nGenerating for QID = {qid} ({idx+1}/{sample_n})")
        pred = generate_answer(question, max_new_tokens=300, temperature=0.2)
    except Exception as e:
        print("Error during generation for QID", qid, ":", e)
        traceback.print_exc()
        pred = f"[ERROR_GENERATING] {e}"

    results.append({
        "questionID": qid,
        "instruction": question,
        "ground_truth": gt,
        "model_prediction": pred
    })

    # طباعة طول results للتأكد
    print("Length of results so far:", len(results))

    # حفظ آمن (اكتب لملف مؤقت ثم ابدله)
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, path)  # atomic replace
    print("Saved", len(results), "records to", path)

    time.sleep(0.4)


Generating for QID = idx_0 (1/5)
Length of results so far: 1
Saved 1 records to sample_comparison.json

Generating for QID = idx_1 (2/5)
Length of results so far: 2
Saved 2 records to sample_comparison.json

Generating for QID = idx_2 (3/5)
Length of results so far: 3
Saved 3 records to sample_comparison.json

Generating for QID = idx_3 (4/5)
Length of results so far: 4
Saved 4 records to sample_comparison.json

Generating for QID = idx_4 (5/5)
Length of results so far: 5
Saved 5 records to sample_comparison.json


In [ ]:
import json
with open("sample_comparison.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print("Final file contains", len(data), "records")

Final file contains 5 records


In [ ]:
# ====== Load file (JSON or JSONL) and compute metrics for the saved samples ======
import json, os
from pathlib import Path

# حاول يقرأ JSONL أولا، بعدين JSON
if Path("sample_comparison.jsonl").exists():
    data = []
    with open("sample_comparison.jsonl", "r", encoding="utf-8") as f:
        for line in f:
            data.append(json.loads(line))
elif Path("sample_comparison.json").exists():
    with open("sample_comparison.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    raise FileNotFoundError("sample_comparison.json(.l) غير موجود. اتأكد من اسم الملف.")

print("Loaded", len(data), "records.")

Loaded 5 records.


In [ ]:
# ====== Install libs if لازم (فقط لو مش منصبة) ======
!pip install -q sacrebleu rouge-score bert-score sentence-transformers numpy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
# ====== Imports للتقييم ======
import numpy as np
import sacrebleu
from rouge_score import rouge_scorer
import bert_score
from sentence_transformers import SentenceTransformer, util

In [ ]:
# ====== إعدادات ======
rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
embedder = SentenceTransformer('all-MiniLM-L6-v2')  # سريع ومتعدد اللغات جيد

refs = [rec['ground_truth'] for rec in data]
preds = [rec['model_prediction'] for rec in data]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# ====== BERTScore دفعة واحدة ======
P, R, F = bert_score.score(preds, refs, lang='en', verbose=False)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ====== Embeddings cosine دفعة واحدة ======
embs_preds = embedder.encode(preds, convert_to_tensor=True, show_progress_bar=False)
embs_refs = embedder.encode(refs, convert_to_tensor=True, show_progress_bar=False)
cosine_batch = util.cos_sim(embs_preds, embs_refs).diagonal().cpu().numpy()

In [ ]:
# ====== حساب المقاييس لكل مثال وتجميع تقرير ======
results = []
for i, rec in enumerate(data):
    ref = refs[i]
    pred = preds[i]
    bleu = sacrebleu.sentence_bleu(pred, [ref]).score if ref and pred else 0.0
    r = rouge.score(ref, pred)
    bert_f = float(F[i].cpu().item())
    cosine = float(cosine_batch[i])
    results.append({
        "questionID": rec.get("questionID"),
        "bleu": bleu,
        "rouge1": r['rouge1'].fmeasure,
        "rouge2": r['rouge2'].fmeasure,
        "rougeL": r['rougeL'].fmeasure,
        "bert_f": bert_f,
        "cosine": cosine
    })

In [ ]:
# ====== ملخص إحصائي ======
def mean(xs): return float(np.mean(xs)) if xs else 0.0
report = {
    "n": len(results),
    "bleu_mean": mean([r['bleu'] for r in results]),
    "rouge1_mean": mean([r['rouge1'] for r in results]),
    "rouge2_mean": mean([r['rouge2'] for r in results]),
    "rougeL_mean": mean([r['rougeL'] for r in results]),
    "bert_f_mean": mean([r['bert_f'] for r in results]),
    "cosine_mean": mean([r['cosine'] for r in results])
}

print("\n=== Summary ===")
for k,v in report.items():
    print(f"{k}: {v}")


=== Summary ===
n: 5
bleu_mean: 1.657020717581025
rouge1_mean: 0.29301722975551786
rouge2_mean: 0.06901644377711276
rougeL_mean: 0.1254957485308378
bert_f_mean: 0.8354787349700927
cosine_mean: 0.6913317561149597


In [ ]:
# ====== احفظ النتائج مع المقاييس في ملف منفصل ======
out_path = "sample_comparison_with_metrics.json"
for i, rec in enumerate(data):
    rec.update(results[i])

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("\nSaved detailed results to", out_path)


Saved detailed results to sample_comparison_with_metrics.json
